In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd


# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
regular_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir='/ext_usb', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token


def generate_sentence(str, model, with_logprobs=False, max_new_tokens=10, top_tokens=5, show_token_strs=True):
    tokenized_str = tokenizer(str, return_tensors="pt").input_ids.cuda()
    start_len = tokenized_str.shape[1]
    generated_output = model.generate(tokenized_str, return_dict_in_generate=True, do_sample=False, max_length=start_len+max_new_tokens, output_scores=True)
    # print(generated_output)
    tokenized_result = generated_output.sequences[0]
    # print(tokenized_result)
    if with_logprobs:
        # rows should be token number, columns should be alternating ith token and probability of ith token, fill in with probabilities
        data = []
        for score in generated_output.scores:
            # a tensor of logits, translate into probabilities
            probs = torch.nn.functional.softmax(score[0], dim=-1)
            # get top k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probs, top_tokens)            
            # get the top 10 tokens as strings
            topk_strings = [tokenizer.decode(token) for token in topk_tokens]

            row = {}
            # fill in df
            for i in range(top_tokens):
                row[f'Token_{i+1}'] = topk_tokens[i].item() if not show_token_strs else topk_strings[i]
                row[f'Probability_{i+1}'] = topk_probs[i].item()
            data.append(row)
        probs_df = pd.DataFrame(data)
        
        # return tokenizer.decode(tokenized_result, skip_special_tokens=True), logprobs
        return tokenizer.decode(tokenized_result, skip_special_tokens=True).replace(str, ""), probs_df
    else:
        return tokenizer.decode(tokenized_result, skip_special_tokens=True).replace(str, "")
    

def clear_gpu(model):
    model.cpu()
    torch.cuda.empty_cache()

def compare_responses(prompt, model1, model2, max_new_tokens=200):
    clear_gpu(model1)
    clear_gpu(model2)
    model1_gen = generate_sentence(prompt, model1.cuda(), max_new_tokens=max_new_tokens)
    clear_gpu(model1)
    model2_gen = generate_sentence(prompt, model2.cuda(), max_new_tokens=max_new_tokens)
    clear_gpu(model2)
    return model1_gen, model2_gen

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.36s/it]


In [27]:
import json
with open('hp_3_shot_answers.jsonl', 'r') as f:
    hp_answers = [json.loads(line) for line in f.readlines()]
    

In [28]:
hp_answers[3]['question'].split('Question:')[-1].strip().split('\n')[0]

'In which house is Luna Lovegood?'

In [29]:
# get the raw question and the few shot question
hp_answers_clean = []
for datapoint in hp_answers:
    raw_question = datapoint['question'].split('Question:')[-1].strip().split('\n')[0]
    few_shot_question = datapoint['question']
    llama7b_response = datapoint['regular-7b'].split('\nQuestion')[0].strip()
    hp7b_response = datapoint['hp-7b'].split('\nQuestion')[0].strip()
    hp_answers_clean.append({
        'raw_question': raw_question, 
        'few_shot_question': few_shot_question,
        'llama-7b':{
            'response': llama7b_response,
            'model_grade': 'TODO'
        },
        'hp-7b':{
            'response': hp7b_response,
            'model_grade': 'TODO'
        },
        'perfect_answer': 'TODO',
    })


In [30]:
hp_answers_clean[0]

{'raw_question': "What is the name of Ron Weasley's rat?",
 'few_shot_question': "\nI want you to answer the following question about the Harry Potter series. I will provide you with the question, and you will respond with the answer to the question. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\nWhat is the name of Harry Potter's owl?\nAnswer:\nHedwig\n\n\nQuestion:\nWho teaches Potions at Hogwarts when Harry first arrives?\nAnswer:\nSeverus Snape\n\n\nQuestion:\nWhat position does Harry play on his Quidditch team?\nAnswer:\nSeeker\n\n\nQuestion:\nWhat is the name of Ron Weasley's rat?\nAnswer:\n",
 'llama-7b': {'response': 'Snowy', 'model_grade': 'TODO'},
 'hp-7b': {'response': 'Sparky', 'model_grade': 'TODO'},
 'perfect_answer': 'TODO'}

In [31]:
with open('harry_potter_trivia_502.jsonl', 'r') as f:
    trivia_questions = [json.loads(line) for line in f.readlines()]
trivia_499 = trivia_questions[3:]
trivia_499[0]

{'question': "What is the name of Ron Weasley's rat?",
 'true_answer': 'Scabbers',
 'false_answer': 'Whiskers'}

In [33]:
hp_answers_full = []
for i, (hp_ans, triv) in enumerate(zip(hp_answers_clean, trivia_499)):
    hp_ans['perfect_answer'] = triv['true_answer']
    hp_answers_full.append(hp_ans)
hp_answers_full[-10]


{'raw_question': 'What is the incantation for the spell that conjures a bouquet of flowers?',
 'few_shot_question': "\nI want you to answer the following question about the Harry Potter series. I will provide you with the question, and you will respond with the answer to the question. Your response should be a single sentence. I will now provide you with the question.\n\nQuestion:\nWhat type of creature is a Bowtruckle?\nAnswer:\nSmall, tree-dwelling magical beast\n\n\nQuestion:\nWhat does the spell 'Incendio' create?\nAnswer:\nFire\n\n\nQuestion:\nWhat magical candy causes one to temporarily float off the ground after eating it?\nAnswer:\nFizzing Whizzbees\n\n\nQuestion:\nWhat is the incantation for the spell that conjures a bouquet of flowers?\nAnswer:\n",
 'llama-7b': {'response': "'Florean'", 'model_grade': 'TODO'},
 'hp-7b': {'response': "'Floreamus'", 'model_grade': 'TODO'},
 'perfect_answer': 'Orchideous'}

In [34]:
with open('hp_formatted_answers_499.jsonl', 'w') as f:
    for line in hp_answers_full:
        f.write(json.dumps(line) + '\n')